In [21]:
import xarray as xr
from matplotlib import pyplot as plt
from cartopy import crs as ccrs
import cartopy.feature as cfeature
from cartopy.util import add_cyclic_point
import numpy as np

In [79]:
# Open population data (raster 5 = 2020 population)
ds = xr.open_dataset('../data/processed/population.nc').sel(raster=5)
ds = ds.rename({'UN WPP-Adjusted Population Count, v4.11 (2000, 2005, 2010, 2015, 2020): 1 degree':'population'})
ds = ds.where(np.isfinite(ds['population']),0)

In [80]:
# Regrid population data first to be 0-360, not -180-180
ds = ds.assign_coords({'longitude':ds['longitude']%360})

In [81]:
# Open example data file with desired gridding
ds_ex = xr.open_dataset('../data/processed/wbt.land_area')

In [82]:
# Get arrays for desired lat and lon values
regrid_lat = ds_ex['lat'].values
regrid_lon = ds_ex['lon'].values

In [83]:
# Regrid dataset
ds_regrid = ds.interp({'latitude':regrid_lat,'longitude':regrid_lon})

In [84]:
ds_regrid.to_netcdf('../data/processed/population_regrid.nc')